In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Eric Meinhardt / emeinhardt@ucsd.edu**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Motivation" data-toc-modified-id="Motivation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Motivation</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Combinatoric-summary" data-toc-modified-id="Combinatoric-summary-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Combinatoric summary</a></span></li></ul></li><li><span><a href="#Identifying-a-possible-Boolean-concept" data-toc-modified-id="Identifying-a-possible-Boolean-concept-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Identifying a possible Boolean concept</a></span></li><li><span><a href="#General-case" data-toc-modified-id="General-case-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>General case</a></span><ul class="toc-item"><li><span><a href="#Demo" data-toc-modified-id="Demo-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Demo</a></span></li></ul></li></ul></div>

# Motivation

In this notebook we show `prague` can be used to identify feature vectors compatible with (or that exactly pick out) an observed set of objects, using phonology as a motivating example.

# Load data

In [2]:
%cd ..

/mnt/cube/home/AD/emeinhar/prague


In [3]:
from funcy import *
from functools import reduce

In [4]:
import numpy as np

In [5]:
import prague

In [6]:
%ls data

bakovic_chart_riggle_hayes_remapped.tsv  brh.npy             hayes_remapped.tsv
bakovic_chart_riggle_hayes.tsv           hayes_features.txt  hayes.tsv
brh_features.txt                         hayes.npy


In [7]:
# objects_in_fp = 'data/hayes.tsv'
# objects_np_in_fp = 'data/hayes.npy'
# feature_list_fp = 'data/hayes_features.txt'

objects_in_fp = 'data/bakovic_chart_riggle_hayes.tsv'
objects_np_in_fp = 'data/brh.npy'
feature_list_fp = 'data/brh_features.txt'

In [8]:
objects = prague.load_objects(objects_in_fp)
len(objects)
objects[:3]

symbols = [o['symbol'] for o in objects]
symbols[:10]

94

[OrderedDict([('symbol', 'p'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '-'),
              ('cont', '-'),
              ('del. rel.', '-'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '0'),
              ('low', '0'),
              ('back', '0'),
              ('front', '0'),
              ('approx', '-'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '0'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'b'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '+'),
              ('cont', '-'),
              ('de

['p', 'b', 'ɸ', 'β', 'f', 'v', 't̪', 'd̪', 'θ', 'ð']

In [9]:
len(objects[0].keys())

24

In [10]:
feature_list = []
with open(feature_list_fp, 'r') as feature_file:
    for feature in feature_file:
        feature_list.append(feature.strip())
len(feature_list)

23

In [11]:
assert len(feature_list) == len(objects[0].keys()) - 1

In [12]:
objects_np = np.load(objects_np_in_fp)
objects_np.shape
objects_np.dtype

(94, 23)

dtype('int8')

In [13]:
assert objects_np.shape[1] == len(feature_list)

In [14]:
assert objects_np.shape[0] == len(objects)

In [15]:
objects[:2]

[OrderedDict([('symbol', 'p'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '-'),
              ('cont', '-'),
              ('del. rel.', '-'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '0'),
              ('low', '0'),
              ('back', '0'),
              ('front', '0'),
              ('approx', '-'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '0'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'b'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '-'),
              ('voice', '+'),
              ('cont', '-'),
              ('de

In [16]:
objects_np[:2]

array([[ 0,  0, -1,  0, -1,  1, -1, -1, -1,  0, -1,  0,  0,  1, -1,  0,
        -1,  0, -1, -1,  0, -1, -1],
       [ 0,  0, -1,  0, -1,  1, -1, -1, -1,  0, -1,  0,  0,  1, -1,  0,
        -1,  0, -1, -1,  0, -1,  1]], dtype=int8)

In [17]:
symbol_to_fv = {o['symbol']:objects_np[i]
                for i,o in enumerate(objects)}

In [18]:
objects_np.shape
unique_objects_np = np.unique(objects_np, axis=0)
unique_objects_np.shape

(94, 23)

(91, 23)

In [19]:
fv_to_symbols_map = {prague.HashableArray(fv):{s for s in symbol_to_fv
                                               if np.array_equal(fv, symbol_to_fv[s])}
                     for fv in unique_objects_np}

def fv_to_symbols(fv):
    return fv_to_symbols_map[prague.HashableArray(fv)]

In [20]:
index_to_symbols = [fv_to_symbols(fv) for fv in unique_objects_np]
index_to_symbols

[{'ħ'},
 {'ʕ'},
 {'ə', 'ɜ'},
 {'ɞ'},
 {'ɛ'},
 {'a'},
 {'œ'},
 {'ɶ'},
 {'ɪ'},
 {'ʏ'},
 {'ʌ'},
 {'ɑ'},
 {'ɔ'},
 {'ɒ'},
 {'ʊ'},
 {'ʈ'},
 {'ɖ'},
 {'ɳ'},
 {'ʧ'},
 {'ʤ'},
 {'ʂ'},
 {'ʐ'},
 {'ʃ'},
 {'ʒ'},
 {'ɽ'},
 {'ɭ'},
 {'c'},
 {'ɟ'},
 {'ɲ'},
 {'ç'},
 {'ʝ'},
 {'h'},
 {'ɦ'},
 {'p'},
 {'b'},
 {'m', 'ɱ'},
 {'f'},
 {'v'},
 {'ɸ'},
 {'β'},
 {'ʔ'},
 {'q'},
 {'ɢ'},
 {'k'},
 {'ɡ'},
 {'ɴ'},
 {'ŋ'},
 {'χ'},
 {'ʁ'},
 {'x'},
 {'ɣ'},
 {'j'},
 {'ʎ'},
 {'ʋ'},
 {'ⱱ'},
 {'ʙ'},
 {'w', 'ɥ'},
 {'ʟ'},
 {'ʀ'},
 {'t'},
 {'d'},
 {'t̪'},
 {'d̪'},
 {'n'},
 {'n̪'},
 {'ʦ'},
 {'ʣ'},
 {'s'},
 {'z'},
 {'θ'},
 {'ð'},
 {'ɹ'},
 {'ɻ'},
 {'ɾ'},
 {'l'},
 {'l̪'},
 {'r'},
 {'ɘ'},
 {'ɐ'},
 {'ɵ'},
 {'ɨ'},
 {'ʉ'},
 {'e'},
 {'æ'},
 {'ø'},
 {'i'},
 {'y'},
 {'ɤ'},
 {'o'},
 {'ɯ'},
 {'u'}]

In [21]:
def extension_to_symbols(extension):
    return np.array(index_to_symbols)[extension.nonzero()[0]]

In [22]:
def symbol_to_fd(symbol):
    return [fd for fd in objects if fd['symbol'] == symbol]

symbol_to_fd('i')

[OrderedDict([('symbol', 'i'),
              ('syll', '+'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '-'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('del. rel.', '0'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '+'),
              ('low', '-'),
              ('back', '-'),
              ('front', '+'),
              ('approx', '+'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '-'),
              ('ATR', '+')])]

In [23]:
feature_list

['ATR',
 'ant',
 'approx',
 'back',
 'c.g.',
 'cons',
 'cont',
 'coronal',
 'del. rel.',
 'dist',
 'dorsal',
 'front',
 'high',
 'labial',
 'lat',
 'low',
 'nas',
 'round',
 's.g.',
 'son',
 'strid',
 'syll',
 'voice']

In [24]:
def pfv_to_fd(pfv):
    return dict(zip(feature_list, pfv))

pfv_to_fd(prague.feature_vector.make_random_pfv(len(feature_list)))

{'ATR': 1,
 'ant': 1,
 'approx': 0,
 'back': 1,
 'c.g.': 0,
 'cons': -1,
 'cont': 0,
 'coronal': 0,
 'del. rel.': -1,
 'dist': 0,
 'dorsal': -1,
 'front': 0,
 'high': 0,
 'labial': 1,
 'lat': 1,
 'low': 0,
 'nas': 0,
 'round': 0,
 's.g.': 1,
 'son': 0,
 'strid': 1,
 'syll': -1,
 'voice': 0}

In [25]:
#more verbose
prague.feature_vector.to_feature_dict(feature_list,
                                      prague.feature_vector.make_random_pfv(len(feature_list)))

{'ATR': '-',
 'ant': '+',
 'approx': '-',
 'back': '0',
 'c.g.': '+',
 'cons': '+',
 'cont': '0',
 'coronal': '-',
 'del. rel.': '0',
 'dist': '+',
 'dorsal': '+',
 'front': '-',
 'high': '-',
 'labial': '0',
 'lat': '-',
 'low': '+',
 'nas': '+',
 'round': '0',
 's.g.': '-',
 'son': '-',
 'strid': '+',
 'syll': '0',
 'voice': '+'}

## Combinatoric summary

In [26]:
f"|O| = {len(objects)} distinct object types (including labels)"
f"{unique_objects_np.shape[1]} object features"
"{0:.2E} logically possible object feature vectors".format(2**unique_objects_np.shape[1])
f"|V| = {unique_objects_np.shape[0]} distinct object feature vectors"
"{0:.2E} possible subsets of V".format(2**unique_objects_np.shape[0])
"{0:.2E} possible partial feature vectors".format(3**unique_objects_np.shape[1])

'|O| = 94 distinct object types (including labels)'

'23 object features'

'8.39E+06 logically possible object feature vectors'

'|V| = 91 distinct object feature vectors'

'2.48E+27 possible subsets of V'

'9.41E+10 possible partial feature vectors'

# Identifying a possible Boolean concept

Consider the two sets of objects defined below:

In [27]:
from random import choice

In [28]:
num_observations = 4
random_observation = [choice(objects) for each in range(num_observations)]
lmap(lambda o: o['symbol'],
     random_observation)
random_observation

['ɣ', 'ɜ', 'ʣ', 'ɭ']

[OrderedDict([('symbol', 'ɣ'),
              ('syll', '-'),
              ('cons', '+'),
              ('son', '-'),
              ('labial', '-'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('del. rel.', '+'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '+'),
              ('low', '-'),
              ('back', '+'),
              ('front', '-'),
              ('approx', '-'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '0'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'ɜ'),
              ('syll', '+'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '-'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('de

In [29]:
# matching_symbols = {'t','d','p','b','k','g'} #will probably consume ALL available memory a few cells downstream
matching_symbols = {'j','w'}
non_random_observation = lfilter(lambda o: o['symbol'] in matching_symbols,
                                 objects)
len(non_random_observation)
non_random_observation

2

[OrderedDict([('symbol', 'w'),
              ('syll', '-'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '+'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('del. rel.', '0'),
              ('ant', '0'),
              ('dist', '0'),
              ('strid', '0'),
              ('high', '+'),
              ('low', '-'),
              ('back', '+'),
              ('front', '-'),
              ('approx', '+'),
              ('nas', '-'),
              ('lat', '-'),
              ('s.g.', '-'),
              ('c.g.', '-'),
              ('round', '+'),
              ('ATR', '0')]),
 OrderedDict([('symbol', 'j'),
              ('syll', '-'),
              ('cons', '-'),
              ('son', '+'),
              ('labial', '-'),
              ('coronal', '-'),
              ('dorsal', '+'),
              ('voice', '+'),
              ('cont', '+'),
              ('de

Suppose we didn't know how either set of observations were generated, but that we want to know if all of the examples in each set of observations are instances of at least one Boolean concept (i.e. plausibly generated by/instances of the same defining partial feature vector).

`prague`'s main functionality is to facilitate this kind of calculation and analysis via two functions:

In [30]:
print(prague.get_pfvs_whose_extension_contains.__doc__)


    Given
        a set of observed objects (a stack of feature vectors)
    this returns
        the set of partial feature vectors (a stack, one vector per row)
    whose extension must contain the set of observed objects.
    


In [31]:
print(prague.get_pfvs_whose_extension_is_exactly.__doc__)


    Given
        a set of observed objects (a stack of feature vectors)
        a set of potentially observable objects (another stack of vectors)
    this returns
        the set of partial feature vectors (a stack, one vector per row)
    whose extension must be exactly the set of observed objects.
    


To use these functions, we first (for each set of observations) map each object to its associated NumPy vector representation:

In [32]:
random_observation_pfvs = np.array([symbol_to_fv[o['symbol']]
                                   for o in random_observation])
random_observation_pfvs

array([[ 0,  0, -1,  1, -1,  1,  1, -1,  1,  0,  1, -1,  1, -1, -1, -1,
        -1,  0, -1, -1,  0, -1,  1],
       [-1,  0,  1, -1, -1, -1,  1, -1,  0,  0,  1, -1, -1, -1, -1, -1,
        -1, -1, -1,  1,  0,  1,  1],
       [ 0,  1, -1,  0, -1,  1, -1,  1,  1, -1, -1,  0,  0, -1, -1,  0,
        -1,  0, -1, -1,  1, -1,  1],
       [ 0, -1,  1,  0, -1,  1, -1,  1,  0, -1, -1,  0,  0, -1,  1,  0,
        -1,  0, -1,  1,  0, -1,  1]], dtype=int8)

Since the random observation set is random from run to run of the notebook, your specific results (if you execute this notebook locally) may vary, but in general, there will be thousands of partial feature vectors (= conjunctive normal form formulas = restricted Boolean concepts) that are compatible with (i.e. could have given rise to) this observation: 

In [33]:
possible_explanations_for_random_observation = prague.get_pfvs_whose_extension_contains(random_observation_pfvs)
possible_explanations_for_random_observation.shape
possible_explanations_for_random_observation

(32, 23)

array([[ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
        -1,  0, -1,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
        -1,  0, -1,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        -1,  0, -1,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
         0,  0, -1,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
        -1,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
        -1,  0, -1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        -1,  0, -1,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
         0,  0, -1,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
        -1,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,

For example:

In [34]:
print(f"Recall: the random observation = \n{lmap(lambda o: o['symbol'], random_observation)}")
a_compatible_concept = choice(possible_explanations_for_random_observation)
print("A possible partial feature vector that could have given rise to this observation:\n"
#       f"{a_compatible_concept}\n"
      "{0}".format({feature_list[i]:{-1:'-',1:'+'}[v] for i,v in enumerate(a_compatible_concept) if v in {1,-1}}))
      
compatible_concept_extension_vector = prague.extension(a_compatible_concept, 
                                                       unique_objects_np)
compatible_concept_extension_as_objects = prague.extension_vector_to_objects(compatible_concept_extension_vector, 
                                                                             unique_objects_np)
compatible_concept_extension_as_symbols = reduce(set.union, 
                                                 lmap(fv_to_symbols, 
                                                      compatible_concept_extension_as_objects), 
                                                 set())
      
print("The total set of observations that exhibit this concept:\n",
      f"{compatible_concept_extension_as_symbols}")

Recall: the random observation = 
['ɣ', 'ɜ', 'ʣ', 'ɭ']
A possible partial feature vector that could have given rise to this observation:
{'labial': '-', 'nas': '-', 's.g.': '-', 'voice': '+'}
The total set of observations that exhibit this concept:
 {'ə', 'ɑ', 'ɜ', 'ʎ', 'z', 'æ', 'ʒ', 'ɢ', 'ɣ', 'l̪', 'ɤ', 'ʁ', 'd̪', 'ɟ', 'a', 'ɽ', 'r', 'ɐ', 'e', 'ʣ', 'ɻ', 'ɖ', 'ʝ', 'ɪ', 'l', 'ɘ', 'ɨ', 'ɭ', 'ʀ', 'd', 'ɡ', 'ʕ', 'ʌ', 'ʊ', 'j', 'ð', 'ʟ', 'ɹ', 'ɯ', 'ʤ', 'ʐ', 'ɾ', 'i', 'ɛ'}


In [35]:
precise_explanations_for_random_observation = prague.get_pfvs_whose_extension_is_exactly(random_observation_pfvs,
                                                                                         objects_np)
precise_explanations_for_random_observation.shape
precise_explanations_for_random_observation

#This will usually be empty: most subsets of the set of logically possible objects
# can't be described by a formula in conjunctive normal form

(0, 23)

array([], shape=(0, 23), dtype=int8)

In [36]:
non_random_observation_pfvs = np.array([symbol_to_fv[o['symbol']]
                                        for o in non_random_observation])
non_random_observation_pfvs

array([[ 0,  0,  1,  1, -1, -1,  1, -1,  0,  0,  1, -1,  1,  1, -1, -1,
        -1,  1, -1,  1,  0, -1,  1],
       [ 0,  0,  1, -1, -1, -1,  1, -1,  0,  0,  1,  1,  1, -1, -1, -1,
        -1, -1, -1,  1,  0, -1,  1]], dtype=int8)

In [37]:
possible_explanations_for_non_random_observation = prague.get_pfvs_whose_extension_contains(non_random_observation_pfvs)
possible_explanations_for_non_random_observation.shape
possible_explanations_for_non_random_observation

(16384, 23)

array([[ 0,  0,  1, ...,  0, -1,  1],
       [ 0,  0,  0, ...,  0, -1,  1],
       [ 0,  0,  1, ...,  0, -1,  1],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  1, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int8)

In [38]:
precise_explanations_for_non_random_observation = prague.get_pfvs_whose_extension_is_exactly(non_random_observation_pfvs,
                                                                                             objects_np)
precise_explanations_for_non_random_observation.shape
precise_explanations_for_non_random_observation

(4672, 23)

array([[ 0,  0,  1, ...,  0, -1,  1],
       [ 0,  0,  0, ...,  0, -1,  1],
       [ 0,  0,  1, ...,  0, -1,  1],
       ...,
       [ 0,  0,  0, ...,  0, -1,  0],
       [ 0,  0,  0, ...,  0, -1,  0],
       [ 0,  0,  0, ...,  0, -1,  0]], dtype=int8)

In [39]:
degree_of_specification_of_exact_matches = np.abs(precise_explanations_for_non_random_observation).sum(axis=1)
minimal_specification_of_exact_matches = np.min(degree_of_specification_of_exact_matches)
minimal_specification_of_exact_matches
minimal_pfvs_that_are_exact_matches = precise_explanations_for_non_random_observation[degree_of_specification_of_exact_matches == minimal_specification_of_exact_matches]
minimal_pfvs_that_are_exact_matches
minimal_pfvs_that_are_exact_matches.shape

for each_pfv in minimal_pfvs_that_are_exact_matches:
    print(pfv_to_fd(each_pfv))
    
for each_pfv in minimal_pfvs_that_are_exact_matches:
    each_ext = prague.extension(each_pfv, unique_objects_np)
    extension_to_symbols(each_ext)

3

array([[ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,
         0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
         0,  0,  0,  0,  0, -1,  0],
       [ 0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, -1,  0]], dtype=int8)

(3, 23)

{'ATR': 0, 'ant': 0, 'approx': 0, 'back': 0, 'c.g.': 0, 'cons': -1, 'cont': 0, 'coronal': 0, 'del. rel.': 0, 'dist': 0, 'dorsal': 0, 'front': 0, 'high': 0, 'labial': 0, 'lat': 0, 'low': -1, 'nas': 0, 'round': 0, 's.g.': 0, 'son': 0, 'strid': 0, 'syll': -1, 'voice': 0}
{'ATR': 0, 'ant': 0, 'approx': 0, 'back': 0, 'c.g.': 0, 'cons': -1, 'cont': 0, 'coronal': 0, 'del. rel.': 0, 'dist': 0, 'dorsal': 0, 'front': 0, 'high': 1, 'labial': 0, 'lat': 0, 'low': 0, 'nas': 0, 'round': 0, 's.g.': 0, 'son': 0, 'strid': 0, 'syll': -1, 'voice': 0}
{'ATR': 0, 'ant': 0, 'approx': 0, 'back': 0, 'c.g.': 0, 'cons': -1, 'cont': 0, 'coronal': 0, 'del. rel.': 0, 'dist': 0, 'dorsal': 1, 'front': 0, 'high': 0, 'labial': 0, 'lat': 0, 'low': 0, 'nas': 0, 'round': 0, 's.g.': 0, 'son': 0, 'strid': 0, 'syll': -1, 'voice': 0}


array([{'j'}, {'w', 'ɥ'}], dtype=object)

array([{'j'}, {'w', 'ɥ'}], dtype=object)

array([{'j'}, {'w', 'ɥ'}], dtype=object)

# General case

In [40]:
from tqdm import tqdm

In [41]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           125G        1.5G         92G        3.1M         31G        123G
Swap:          2.0G          0B        2.0G


In [44]:
#7s on wittgenstein
lower_closures = [prague.lower_closure(o, strict=False) 
                  for o in tqdm(unique_objects_np)]

100%|██████████| 91/91 [00:07<00:00,  8.21it/s]


In [45]:
lower_closures_as_matrix = np.concatenate(lower_closures)

In [46]:
#2m on wittgenstein
all_pfvs_with_nonempty_extension = np.unique(lower_closures_as_matrix, 
                                             return_index=False,
                                             axis=0)

In [47]:
all_pfvs_with_nonempty_extension.shape

(9115112, 23)

In [48]:
print("# partial feature vectors that pick out a non-empty subset of O: "
      "{0:.2E}\n".format(all_pfvs_with_nonempty_extension.shape[0]),
      "# logically possible partial feature vectors: "
      "{0:.2E}".format(3**all_pfvs_with_nonempty_extension.shape[1]))

# partial feature vectors that pick out a non-empty subset of O: 9.12E+06
 # logically possible partial feature vectors: 9.41E+10


In [49]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           125G        2.8G         91G        3.1M         31G        121G
Swap:          2.0G          0B        2.0G


In [47]:
# 2m on wittgenstein
# unique_lower_closures = set(lmap(prague.HashableArray,
#                                  list(lower_closures_as_matrix)))

In [50]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           125G        2.8G         91G        3.1M         31G        121G
Swap:          2.0G          0B        2.0G


In [49]:
# >>>2m on wittgenstein, uses way too much memory
# unique_lower_closures2 = set(lmap(prague.HashableArray,
#                                   cat(map(list, lower_closures_as_matrix))))

In [51]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           125G        2.8G         91G        3.1M         31G        121G
Swap:          2.0G          0B        2.0G


In [52]:
#48s on wittgenstein
nonempty_pfv_extensions = prague.extensions(all_pfvs_with_nonempty_extension,
                                            object_inventory=unique_objects_np)

In [53]:
nonempty_pfv_extensions.shape

(9115112, 91)

In [54]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           125G        3.6G         90G        3.1M         31G        121G
Swap:          2.0G          0B        2.0G


## Demo

In [55]:
a_random_pfv = prague.feature_vector.make_random_pfv(num_features=len(feature_list))
random_extension = prague.extension(a_random_pfv, 
                                    object_inventory=unique_objects_np)
random_objects = prague.extension_vector_to_objects(random_extension, 
                                                    object_inventory=unique_objects_np)
# random_symbols = extension_to_symbols(random_extension)
random_symbols = reduce(set.union, extension_to_symbols(random_extension), set())
my_x = random_extension
my_S = random_objects
my_symbs = random_symbols
my_fd = pfv_to_fd(a_random_pfv)
my_symbs_fds = [pfv_to_fd(pfv) for pfv in my_S]


while my_x.sum() == 0:
    a_random_pfv = prague.feature_vector.make_random_pfv(num_features=len(feature_list))
    random_extension = prague.extension(a_random_pfv, 
                                        object_inventory=unique_objects_np)
    random_objects = prague.extension_vector_to_objects(random_extension, 
                                                        object_inventory=unique_objects_np)
#     random_symbols = extension_to_symbols(random_extension)
    random_symbols = reduce(set.union, extension_to_symbols(random_extension), set())
    my_x = random_extension
    my_S = random_objects
    my_symbs = random_symbols
    my_fd = pfv_to_fd(a_random_pfv)
    my_symbs_fds = [pfv_to_fd(pfv) for pfv in my_S]


print(f"My random PFV = {a_random_pfv}")
print(f"My random PFV as a feature dict =\n{my_fd}")
print(f"My extension as an 'indicator' vector = \n{my_x}")
print(f"My extension as a stack of {my_S.shape[0]} objects S = \n{my_S}")
print(f"My extension as a set of matching symbols = \n{my_symbs}")
print(f"My extension as a set of feature dicts = \n",
      my_symbs_fds)

My random PFV = [ 0  0  0  0 -1  0  1 -1  0  0  0 -1  1  1  0  0 -1  0  0  0  0  0  0]
My random PFV as a feature dict =
{'ATR': 0, 'ant': 0, 'approx': 0, 'back': 0, 'c.g.': -1, 'cons': 0, 'cont': 1, 'coronal': -1, 'del. rel.': 0, 'dist': 0, 'dorsal': 0, 'front': -1, 'high': 1, 'labial': 1, 'lat': 0, 'low': 0, 'nas': -1, 'round': 0, 's.g.': 0, 'son': 0, 'strid': 0, 'syll': 0, 'voice': 0}
My extension as an 'indicator' vector = 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]
My extension as a stack of 3 objects S = 
[[ 0  0  1  1 -1 -1  1 -1  0  0  1 -1  1  1 -1 -1 -1  1 -1  1  0 -1  1]
 [ 1  0  1 -1 -1 -1  1 -1  0  0  1 -1  1  1 -1 -1 -1  1 -1  1  0  1  1]
 [ 1  0  1  1 -1 -1  1 -1  0  0  1 -1  1  1 -1 -1 -1  1 -1  1  0  1  1]]
My extension as a set of matching symbols = 
{'u', 'w', 'ɥ', 'ʉ'}
My extension as a set of feature dicts = 
 [{'ATR': 0, 'a

In [56]:
pfvs_containing_my_S = prague.get_pfvs_whose_extension_contains(my_S)
pfvs_exactly_matching_my_S = prague.get_pfvs_whose_extension_is_exactly(my_S, 
                                                                        object_inventory=unique_objects_np)
specification_of_exact_matches = np.abs(pfvs_exactly_matching_my_S).sum(axis=1)
minimal_specification = np.min(specification_of_exact_matches)
minimal_pfvs_exactly_matching_my_s = pfvs_exactly_matching_my_S[specification_of_exact_matches == minimal_specification]

print("{0:,} PFVs".format(pfvs_containing_my_S.shape[0]),
      f"whose extension contains S = \n{pfvs_containing_my_S}")
print("{0:,} PFVs".format(pfvs_exactly_matching_my_S.shape[0]),
      f"whose extension is exactly S = \n{pfvs_exactly_matching_my_S}")
print("{0:,} PFVs".format(minimal_pfvs_exactly_matching_my_s.shape[0]),
      f"whose extension is exactly S and which are maximally simple"
      f" (i.e. unspecified) = \n{minimal_pfvs_exactly_matching_my_s}")
print(f"original generating pfv = \n {a_random_pfv}")

65,536 PFVs whose extension contains S = 
[[0 0 1 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
12,288 PFVs whose extension is exactly S = 
[[0 0 1 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 1 ... 0 0 1]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
2 PFVs whose extension is exactly S and which are maximally simple (i.e. unspecified) = 
[[ 0  0  0  0  0  0  0  0  0  0  0 -1  1  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 -1  1  1  0  0  0  0  0  0  0  0  0]]
original generating pfv = 
 [ 0  0  0  0 -1  0  1 -1  0  0  0 -1  1  1  0  0 -1  0  0  0  0  0  0]


In [57]:
print(prague.extension(a_random_pfv, unique_objects_np))
for each_exact_match in minimal_pfvs_exactly_matching_my_s:
    print(prague.extension(each_exact_match, unique_objects_np))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1]


In [58]:
extension_to_symbols(prague.extension(a_random_pfv, unique_objects_np))
for each_exact_match in minimal_pfvs_exactly_matching_my_s:
    extension_to_symbols(prague.extension(each_exact_match, unique_objects_np))

array([{'w', 'ɥ'}, {'ʉ'}, {'u'}], dtype=object)

array([{'w', 'ɥ'}, {'ʉ'}, {'u'}], dtype=object)

array([{'w', 'ɥ'}, {'ʉ'}, {'u'}], dtype=object)